# Haiku Generation GAN fine-tuning w/ fine-tuned Generator

In [1]:
from transformers import BartTokenizer, BartModel,BartForConditionalGeneration
from torch.distributions import Categorical
import pandas as pd
import torch.nn as nn
from undecorated import undecorated
import numpy as np
from datasets import load_metric
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader
from transformers import  Seq2SeqTrainingArguments, Seq2SeqTrainer,DataCollatorForSeq2Seq
metric = load_metric("rouge")
import torch.optim as optim
import torchvision.utils as vutils
import types
import matplotlib.pyplot as plt
from torch.distributions import Categorical
import random
#from torch.linalg import norm

<ipython-input-1-a068916ada98>:12: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [ ]:
from gan_utils import get_end_mask,get_valid_mask,Discriminator,discriminator_train_standard, generate_random_input,reinforce_loss,Generator


In [2]:
from utils import PoemDataset,encode_sentences




In [3]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
#generator = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

# Data


In [8]:
processed_data_dir =  '/home/alexander/nlg-project/data/'
test_df = pd.read_csv(processed_data_dir + 'test_data.csv')
test_model = encode_sentences(tokenizer,test_df)

train_df = pd.read_csv(processed_data_dir + 'train_data.csv')
train_model = encode_sentences(tokenizer,train_df)

In [17]:
#logprob

In [28]:
#_,logprob = simple_decoding(generator,noise)

In [13]:
train_ds = PoemDataset(train_model)
eval_ds = PoemDataset(test_model)

In [30]:
#discriminator = discriminator.cuda()

# Train

In [31]:
#fixed_noise = generate_random_input(bsize,'cuda')

In [14]:
PATH = '/home/alexander/nlg-project/GAN_models_finetuned/'






In [15]:
 #reduced to a binary problem 
bsize = 12
generator =  Generator().cuda()#BartForConditionalGeneration.from_pretrained('facebook/bart-base').cuda()

#first time was loaded from finetuned BART 
generator.load_state_dict(torch.load(PATH + 'generator.pt')['model_state_dict'])
discriminator = Discriminator().cuda()
discriminator.load_state_dict(torch.load(PATH + 'discriminator.pt')['model_state_dict'])
criterion = nn.BCEWithLogitsLoss(reduction='none')

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = generate_random_input(bsize,'cuda')

# Establish convention for real and fake labels during training
real_label = 1.
ewma_reward = 0.0#torch.tensor(0.0)
fake_label = 0.
lr = .000001
beta1 = .9
# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(discriminator.parameters(), lr=lr*.05, betas=(beta1, 0.99)) 
optimizerG = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.99))




In [21]:
ls

'=20.9'                     GAN_models/             data/
 BART-GAN-finetuned.ipynb   GAN_models_finetuned/   data_processing.ipynb
 BART-GAN.ipynb             G_3epochs.txt           data_raw/
 BART-base/                 G_losses_2.txt          finetuned_bart.ipynb
 D_3epochs.txt              G_losses_3.txt          utils.py
 D_losses_2.txt             __init__.py
 D_losses_3.txt             __pycache__/


In [33]:
#fixed_noise

In [34]:
#out.sequences

In [22]:


with open("g_loss_finetuned_four.txt", "w") as f:
    for s in G_losses:
        f.write(str(s) +"\n")

In [23]:


with open("d_loss_finetuned_four.txt", "w") as f:
    for s in D_losses:
        f.write(str(s) +"\n")

In [16]:
#this code has been heavily modified, but based on https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
#not copies though so id 
# Training Loop
device = 'cuda'

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0
num_epochs = 30

train_dl = DataLoader(train_ds,batch_size=bsize,num_workers=2)
print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    print("Label Noise:",0.05)
    # For each batch in the dataloader
    for i, data in enumerate(train_dl, 0):
       # print(i)
        
        if len(data['input_ids']) < bsize:
            continue
       # print(data)
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        
        # Format batch
        real_inputs = data['input_ids'].to(device)
        real_input_attention = data['attention_mask'].to(device)
        real_input_test = {'input_ids':real_inputs,'attention_mask':real_input_attention}
        
        
        real_gpu = data['label_ids'].to(device)
        attention_gpu = data['decoder_attention_mask'].to(device)
        b_size = real_gpu.size(0)
        #
        label = torch.full((b_size,), 1.0 , dtype=torch.float, device=device) #.95 instead of real to inject noise
        label = torch.bernoulli(label)
       # print(label)
        # Forward pass real batch through D
        real_mask = get_valid_mask(real_gpu)

        output_real,loss_real = discriminator_train(real_gpu[:,1:],label,criterion,mask=real_mask)

        loss_real.backward()

        #del label
        #output = discriminator(real_gpu,attention_mask=attention_gpu).view(-1)
        # Calculate loss on all-real batch
        #errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        #errD_real.backward()
        D_x = output_real.detach().mean().item()
        #del output_real
        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = generate_random_input(bsize,device)#torch.randn(b_size, nz, 1, 1, device=device)
        # Generate fake image batch with G
        #print(fake)
       # print(noise)
        if random.random() > .5:

            fake,fake_logprobs = generator(noise)
        else:

            #sometimes the discirminator learns words in real input, this way some will be based on these same words
            fake,fake_logprobs = generator(real_input_test)
            #real_input_test
  
        
        fake_mask = get_valid_mask(fake)
        end_mask = get_end_mask(fake)
  
        #generate_sequence(generator,noise)
       # print(len(fake_logprobs))
        #a = 1/0
        #fake = generator.generate_with_grad(noise['input_ids'].cuda(),attention_mask=noise["attention_mask"].cuda()) #,max_length=128
       # print(fake)
        #print((fake.logits))
        #fake = decode(fake.logits)
      #  print(fake)
        # I am injecting some false positives into the fake labels so the discriminator does a worse job
        label.fill_(0.00) 
        label = torch.bernoulli(label)
       # print(label)
        # Classify all fake batch with D
        output_fake,loss_fake = discriminator_train(fake,label,criterion,mask=fake_mask)

        loss_fake.backward()
       # print('fake')
       # print(loss_fake.detach())
        #output = discriminator(fake).view(-1)
        # Calculate D's loss on the all-fake batch
        #errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
       # errD_fake.backward()
        D_G_z1 = output_fake.detach().cpu().mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = (torch.mean(loss_real.detach()) + torch.mean(loss_fake.detach())).cpu()
       # del output_real
        # Update D
        optimizerD.step()
        

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        
        #generator.zero_grad()

        lossG, cumulative_rewards, ewma_reward = reinforce_loss(output_fake,fake_logprobs,.3,.08,epoch,fake_mask,end_mask)
        if i % 256 == 0:
            print('Disc LogProbs True')
            print(output_real[0:1])
            print('Disc LogProbs False')
            print(output_fake[0:1])
            print('gen loss')
            print(lossG[0:1])
            print('cum rewards')
            print(cumulative_rewards[0:1])
            print('gen logprobs')
            print(fake_logprobs[0:1])
            print(tokenizer.batch_decode(fake[0:1].cpu()))

        #print(lossG)
        lossG = torch.mean(lossG)
       # print(lossG)
        #print(loss)
        #label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
       # print(fake)
        #output = discriminator(fake).view(-1)
        # Calculate G's loss based on this output
        #errG = criterion(output, label)
        #print(errG)
        #errG = loss.detach()
        # Calculate gradients for G
        lossG.backward()
        D_G_z2 = lossG.detach().cpu().mean().item()
        # Update G
        
        optimizerG.step()
        generator.zero_grad()
        discriminator.zero_grad()
        #a = 1/0
        # Output training stats
        if i % 128 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(train_dl),
                     errD.item(), lossG.item(), D_x, D_G_z1, D_G_z2))
          #  print('ewma_reward:',ewma_reward)

        # Save Losses for plotting later
        G_losses.append(lossG.detach().item())
        D_losses.append(errD.detach().item())
        
        del lossG
        del errD

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = tokenizer.batch_decode(generator.model.generate(fixed_noise['input_ids'],attention_mask=fixed_noise["attention_mask"],max_length=32,num_beams= 50,early_stopping = True))
                print(fake[0:2])
        if (iters % 1000 == 0 ):
            print('saving geneartor')
            torch.save({
            'epoch': 15,
            'model_state_dict': generator.state_dict(),
            'optimizer_state_dict': optimizerG.state_dict(),
            'loss': 0.3355,
            }, PATH + 'generator.pt')
            torch.save({
            'epoch': 15,
            'model_state_dict': discriminator.state_dict(),
            'optimizer_state_dict': optimizerD.state_dict(),
            'loss': 0.3355,
            }, PATH + 'discriminator.pt')
            #img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

Starting Training Loop...
Label Noise: 0.05


<ipython-input-7-f6867a44ab39>:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output_tokens = torch.cat([output_tokens,torch.tensor(next_word).unsqueeze(1)],axis=1)
/home/alexander/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Disc LogProbs True
tensor([[-0.5322, -0.2014, -0.4056, -0.5340, -0.9182, -0.0259,  0.1825,  0.5351,
          1.3155,  1.3597,  1.6611,  2.1858,  1.7843,  2.2845,  2.4338,  2.4559,
          2.7625,  2.9814,  3.0954,  3.2381,  3.1147]], device='cuda:0',
       grad_fn=<SliceBackward>)
Disc LogProbs False
tensor([[ 0.2824, -0.2379, -0.3094, -0.0000, -0.0000,  0.0000, -0.0000, -0.0000,
         -0.0000, -0.0000, -0.0000, -0.0000, -0.0000,  0.0000,  0.0000]],
       device='cuda:0', grad_fn=<SliceBackward>)
gen loss
tensor([[ 0.0236, -0.0232, -0.0266,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],
       device='cuda:0', grad_fn=<SliceBackward>)
cum rewards
tensor([[ 0.1403, -0.1539, -0.2133,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],
       device='cuda:0', grad_fn=<SliceBackward>)
gen logprobs
tensor([[-1.2456e-01, -2.2157e-01, -1.6169e-01, 

[0/30][1280/11403]	Loss_D: 1.5200	Loss_G: 0.0000	D(x): 2.7161	D(G(z)): 0.0000 / 0.0000
[0/30][1408/11403]	Loss_D: 1.1352	Loss_G: 0.0000	D(x): 1.9956	D(G(z)): 0.0000 / 0.0000
['</s><s> use each\nless than yom i will neither\nspo my<pad></s>', '</s><s> use each\nless than yom i will neither\nspo my<pad></s>']
Disc LogProbs True
tensor([[2.4995, 2.8206, 2.6507, 3.0191, 3.1966, 3.3760, 3.4891, 3.7785, 3.8598,
         3.9574, 4.0414, 4.0705, 4.0805, 4.1356, 4.1345, 4.0306, 4.0808, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<SliceBackward>)
Disc LogProbs False
tensor([[0., 0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., 0., 0., 0.]],
       device='cuda:0', grad_fn=<SliceBackward>)
gen loss
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -0., -0., -0., -0.]],
       device='cuda:0', grad_fn=<SliceBackward>)
cum rewards
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       device='cuda:

[0/30][2816/11403]	Loss_D: 12.0616	Loss_G: 0.0580	D(x): 1.4555	D(G(z)): -0.4115 / 0.0580
[0/30][2944/11403]	Loss_D: 9.7987	Loss_G: -0.0087	D(x): 2.3776	D(G(z)): -0.8134 / -0.0087
['</s><s> useless\nless power neither i\nand neither i y</s><pad><pad>', '</s><s> useless\nless power neither i\nand neither i y</s><pad><pad>']
saving geneartor
Disc LogProbs True
tensor([[1.8828, 1.6833, 2.2605, 3.0096, 3.4158, 3.5884, 3.7006, 3.7374, 3.8502,
         3.8945, 3.9311, 3.9304, 3.6714, 3.6935, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000]], device='cuda:0', grad_fn=<SliceBackward>)
Disc LogProbs False
tensor([[ 1.0998,  0.4729,  0.1700, -0.1543, -1.2760, -1.7594, -2.0341, -2.2801,
         -2.4399, -2.4882, -2.6513, -2.5967, -2.6213, -2.7269, -2.8766]],
       device='cuda:0', grad_fn=<SliceBackward>)
gen loss
tensor([[ 1.6438,  0.4853,  1.0187,  0.7222,  0.0333, -0.2469, -0.3325, -0.3880,
         -0.4993, -0.5086, -0.6201, -3.4728, -0.4312, -0.4304, -1.1457]],
       device='cuda:0', grad_

[0/30][4352/11403]	Loss_D: 13.7675	Loss_G: -0.3609	D(x): 2.6887	D(G(z)): -0.8959 / -0.3609
[0/30][4480/11403]	Loss_D: 28.7499	Loss_G: 0.5592	D(x): 0.9064	D(G(z)): 0.0607 / 0.5592
['</s><s> use your power\nto study but never use\nit for research</s><pad><pad><pad>', '</s><s> used to suck my\njuice but i forgot to\nuse my power</s><pad>']
Disc LogProbs True
tensor([[ 0.9649,  0.0750, -0.9509,  1.6573,  0.5589, -0.0456, -0.1080,  0.0758,
         -0.3541, -0.9003, -1.0902, -1.3983, -1.7294, -1.6264, -1.4490,  0.5767,
          3.3696,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000,
         -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000,
         -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000,
         -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000,  0.0000]],
       device='cuda:0', grad_fn=<SliceBackward>)
Disc LogProbs False
tensor([[ 1.1

Disc LogProbs True
tensor([[-0.4232,  0.2895,  1.5050,  1.0786,  2.9776,  3.2056,  3.2262,  3.0468,
          3.0888,  2.8100,  3.0642,  2.9066,  2.9773,  3.5387,  3.5956,  3.6615,
          3.7101,  3.8644,  3.5356,  3.5902,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000]], device='cuda:0',
       grad_fn=<SliceBackward>)
Disc LogProbs False
tensor([[-0.0000, -0.3649,  0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000,
         -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000,  0.0000]],
       device='cuda:0', grad_fn=<SliceBackward>)
gen loss
tensor([[ 0.0000, -0.0088,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000]],
       device='cuda:0', grad_fn=<SliceBackward>)
cum rewards
tensor([[ 0.0000, -0.2346,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],
       device='cuda:0', grad_fn=<Slice

[0/30][7168/11403]	Loss_D: 0.3680	Loss_G: 0.0000	D(x): 3.4282	D(G(z)): 0.0000 / 0.0000
[0/30][7296/11403]	Loss_D: 0.3250	Loss_G: 0.0000	D(x): 3.6913	D(G(z)): 0.0000 / 0.0000
Disc LogProbs True
tensor([[4.2690, 2.0657, 3.1312, 3.3598, 3.5582, 3.7066, 3.9482, 4.0602, 4.1276,
         4.1919, 4.2758, 4.2900, 4.3576, 4.3440, 4.3543, 4.3664, 4.3652, 4.4150,
         4.3587, 4.3027, 4.3372, 0.0000, 0.0000, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<SliceBackward>)
Disc LogProbs False
tensor([[0., 0., 0., 0., 0., 0., -0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       device='cuda:0', grad_fn=<SliceBackward>)
gen loss
tensor([[0., 0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.]],
       device='cuda:0', grad_fn=<SliceBackward>)
cum rewards
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       device='cuda:0', grad_fn=<SliceBackward>)
gen logprobs
tensor([[-5.7220e-06, -5.7220e-06,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.00

KeyboardInterrupt: 

early poem

> leaving later\nin between the Court of justice\nand the Wander in'

' we what\nwould destiny do if she\ndied and did not live'

 everyone easter\nthe patient waits for the light\nto dim   (wish it was every...)

sampling from latent space sometimes never really impvoes

 you should be writing\nlike an old man in love with\nan old man

epoch after epoch

# Save models

In [21]:
torch.save({
            'epoch': 15,
            'model_state_dict': generator.state_dict(),
            'optimizer_state_dict': optimizerG.state_dict(),
            'loss': 0.3355,
            }, PATH + 'four_it_ft_generator_bad.pt')
torch.save({
            'epoch': 15,
            'model_state_dict': discriminator.state_dict(),
            'optimizer_state_dict': optimizerD.state_dict(),
            'loss': 0.3355,
            }, PATH + 'four_ft_discriminator_bad.pt')







In [ ]:
inputs = tokenizer("cats", return_tensors="pt")

# Testing

In [ ]:
batch_size = 16 

args = Seq2SeqTrainingArguments(
    "BART-base",
    evaluation_strategy = "epoch",
    #learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
   # weight_decay=0.01,
   # save_total_limit=3,
   # num_train_epochs=1,
   # predict_with_generate=True,
    
    learning_rate=1e-4,
    weight_decay=0.01,
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-6,
    max_grad_norm=1.0,
    num_train_epochs=10,
    lr_scheduler_type="linear",
    warmup_ratio=0.1,
   # label_names="labels",
    #fp16=True,
  #  use_auth_token=False
    #push_to_hub=True,
)

In [ ]:
#train_ds = PoemDataset(model_input)
trainer = Seq2SeqTrainer(
    model=BartForConditionalGeneration.from_pretrained('BART-base/checkpoint-25000'),
    args=args,
   # data_collator=data_collator,
    train_dataset=train_ds,
    
    eval_dataset=eval_ds,
    #use_auth_token=False,
    
    tokenizer=tokenizer
)

In [ ]:
trainer.evaluate()

In [ ]:
pwd

In [ ]:
mkdir GAN_models

In [ ]:
torch.save({
            'epoch': 15,
            'model_state_dict': generator.state_dict(),
            'optimizer_state_dict': optimizerG.state_dict(),
            'loss': 0.3355,
            }, PATH + 'generator.pt')

In [ ]:
torch.save({
            'epoch': 15,
            'model_state_dict': discriminator.state_dict(),
            'optimizer_state_dict': optimizerD.state_dict(),
            'loss': 0.3355,
            }, PATH + 'discriminator.pt')

In [ ]:
pwd

In [ ]:
cd GAN_models